# Data Extraction and Cleaning - Brazilian Cities Weather Data

This notebook focuses on importing, processing, and cleaning data to be used in the next step. For this, it is necessary to understand how these data are made available by the Brazilian government.

---

Data are provided by the 'Instituto Nacional de Meteorologia', which mean literally National Institute of Meteorology. This data is open and can be found on their website where it is possible to get data since 2000 with hourly registration. The data is in CSV files which can be downloaded in a zip file for each year.

Data can be retrieved at this link: [INMET](https://portal.inmet.gov.br/dadoshistoricos)

In [12]:
import os
import shutil
import urllib
import zipfile

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

# Setting Seaborn Theme
sns.set_theme(style="whitegrid", font_scale=1.4)

plt.rcParams["figure.figsize"] = (12,7)

In [13]:
# Getting Initial Files
initial_files = os.listdir()

For this project, we will select the brazilian city 'Brasilia', Capital of Brazil.

Initially, it will be selected years between 2011 and 2020 for no special reason (arbitrary).

In [28]:
# Setting chosen data
city = 'brasilia'
start_year = 2011
end_year = 2020

# Creating a list of years for iteration
years = np.arange(start_year,end_year+1,1)

# List with strings(path) for the downloaded files
files_path = []

Instead of downloading each zip file manually, it is better to simply download using python.

In [29]:
# Getting Year by Year zip files
for year in years:

    # Using f-strings to select zip files.
    zip_url = f"https://portal.inmet.gov.br/uploads/dadoshistoricos/{year}.zip"

    # Getting original zip file name
    zip_file = zip_url.split('/')[-1]

    # Downloading File
    urllib.request.urlretrieve(zip_url, zip_file)

    # Reading Zip File and getting only the file of the selected city
    with zipfile.ZipFile(zip_file, "r") as f:
        for name in f.namelist():
            if city in name.lower():
              f.extract(name, path=None, pwd=None)
              files_path.append(name)

Here we will need to take a look into one of the excel sheets to understand how data is in the sheets.

In [30]:
test_data = pd.read_csv(files_path[0],sep=';',header=8, encoding='latin-1')
test_data.head()

,DATA (YYYY-MM-DD),HORA (UTC),"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (KJ/m²),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)",Unnamed: 19
0,2011-01-01,00:00,0,"881,8","881,8",881,-9999,"19,2","18,3","19,4","19,2","18,5","18,3",95,94,95,342,"5,8","3,4",NaN
1,2011-01-01,01:00,0,"882,4","882,4","881,7",-9999,"19,3","18,2","19,3","19,2","18,4","18,2",95,93,93,334,"5,6","2,2",NaN
2,2011-01-01,02:00,0,"882,7","882,7","882,4",-9999,"19,1","18,3","19,3","19,1","18,3","18,2",95,93,95,347,"5,6",3,NaN
3,2011-01-01,03:00,6,"882,6","882,9","882,6",",3","18,1","17,6","19,1","18,1","18,4","17,6",97,95,96,310,"7,9","3,7",NaN
4,2011-01-01,04:00,3,882,"882,6",882,-9999,"17,7","17,2","18,2","17,7","17,6","17,1",97,96,96,335,"6,7","2,4",NaN


With a quick look we can see that:

- Numbers are using a comma
- Date and hour are strings
- the value '-9999' is probably an error or NaN values


It will not be shown here, but some files have different header string but data in the same column index have the same meaning at all files, so it will be necessary to select data needed and rename with a standard name.

In [31]:
# Showing all columns and indexes
print(* (f"{i}: {col}" for i,col in enumerate(temp_df.columns)), sep='\n')

0: DATA (YYYY-MM-DD)
1: HORA (UTC)
2: PRECIPITAÇÃO TOTAL, HORÁRIO (mm)
3: PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)
4: PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)
5: PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)
6: RADIACAO GLOBAL (KJ/m²)
7: TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)
8: TEMPERATURA DO PONTO DE ORVALHO (°C)
9: TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)
10: TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)
11: TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)
12: TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)
13: UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)
14: UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)
15: UMIDADE RELATIVA DO AR, HORARIA (%)
16: VENTO, DIREÇÃO HORARIA (gr) (° (gr))
17: VENTO, RAJADA MAXIMA (m/s)
18: VENTO, VELOCIDADE HORARIA (m/s)
19: Unnamed: 19


Here we will select:
- 0: Date
- 1: Hour
- 3: Atmospheric Pressure
- 7: Temperature

These are the data we are using for this notebook, but there are some interesting data like humidity, radiation and wind speed.

In [32]:
# Selecting and setting data
columns_selected = [0,1,3,7]
test_data = test_data[test_data.columns[columns_selected]]

In [33]:
actual_columns = test_data.columns.to_list()

# Creating a dictionary to help us renaming columns
new_columns = {}
new_columns[actual_columns[0]] = 'date'
new_columns[actual_columns[1]] = 'hour'
new_columns[actual_columns[2]] = 'p (mB)'
new_columns[actual_columns[3]] = 'Temperature'

# Using Dict to rename columns
test_data.rename(columns=new_columns, inplace=True)

In [34]:
test_data.head()

,date,hour,p (mB),Temperature
0,2011-01-01,00:00,"881,8","19,2"
1,2011-01-01,01:00,"882,4","19,3"
2,2011-01-01,02:00,"882,7","19,1"
3,2011-01-01,03:00,"882,6","18,1"
4,2011-01-01,04:00,882,"17,7"


Now we need to handle date, time and numbers that are strings but there is a catch:

- 'hour' data have different formats through years, but since it is hourly data I will use a not fancy solution but that will work around this problem.

In [35]:
# Fixing date and hour problems
test_data['date'] = test_data['date'].str.replace('/','-')
test_data['hour'] = test_data['hour'].apply(lambda x: x[:2])

# Creating a datetime column to use as datetime index
test_data['datetime'] = test_data['date'] + ' ' + test_data['hour']
test_data['datetime'] = pd.to_datetime(test_data['datetime'], format='%Y-%m-%d %H')

# Setting datetime index
test_data.drop(['date','hour'],axis=1,inplace=True)
test_data.set_index('datetime',inplace=True)

# Fixing numbers with comma

# Pressure Data
test_data['p (mB)'] = test_data['p (mB)'].str.replace(',','.')
test_data['p (mB)'] = test_data['p (mB)'].astype(float)

# Temperature Date
test_data['Temperature'] = test_data['Temperature'].str.replace(',','.')
test_data['Temperature'] = test_data['Temperature'].astype(float)


# Making '-9999' numbers a nan
test_data = test_data.replace(-9999.0, np.nan)

In [37]:
# Check NaN Values
test_data.isnull().sum() / len(test_data)

p (mB)         0.001027
Temperature    0.001027
dtype: float64

0.1% of data are missing values or errors. Since it is few datapoints missing we will interpolate to fill this data.

In [38]:
test_data['p (mB)'] = test_data['p (mB)'].interpolate()
test_data['Temperature'] = test_data['Temperature'].interpolate()

In [39]:
test_data.head()

,p (mB),Temperature
datetime,,
2011-01-01 00:00:00,881.8,19.2
2011-01-01 01:00:00,882.4,19.3
2011-01-01 02:00:00,882.7,19.1
2011-01-01 03:00:00,882.6,18.1
2011-01-01 04:00:00,882.0,17.7


It looks all set, from here we can create functions that will do all the steps necessary to use this data.

In [44]:
def get_data(city,start_year,end_year):
    """
    input:
    city: str, selected city
    start_year: int
    end_year: int

    output:
    files_path: List[str], list containing csv file paths
    """

    years = np.arange(start_year,end_year+1,1)
    files_path = []

    for year in years:

        zip_url = f"https://portal.inmet.gov.br/uploads/dadoshistoricos/{year}.zip"
        zip_file = zip_url.split('/')[-1]
        urllib.request.urlretrieve(zip_url, zip_file)

        with zipfile.ZipFile(zip_file, "r") as f:
            for name in f.namelist():
                if city in name.lower():
                  print(f"{city.upper}\nYear:{year}\n_____________________________________")
                  f.extract(name, path=None, pwd=None)
                  files_path.append(name)

    return files_path

def read_file(file):
    """
    Get csv string path and returns pandas dataframe with columns selected
    and columns renamed
    """

    temp_df = pd.read_csv(file,sep=';',header=8, encoding='latin-1')

    # Selecting and setting data
    columns_selected = [0,1,3,7]
    temp_df = temp_df[temp_df.columns[columns_selected]]

    actual_columns = temp_df.columns.to_list()

    # Creating a dictionary to help us renaming columns
    new_columns = {}

    new_columns[actual_columns[0]] = 'date'
    new_columns[actual_columns[1]] = 'hour'
    new_columns[actual_columns[2]] = 'p (mB)'
    new_columns[actual_columns[3]] = 'Temperature'

    # Using Dict to rename columns
    temp_df.rename(columns=new_columns, inplace=True)

    return temp_df

def create_dataframe(csvs_path_list):
    """
    Gets a list of strings containing csv files path and create a dataframe 
    with all data from files combined. Clean and format data.
    """

    df = pd.concat([read_file(file) for file in csvs_path_list], axis=0)

    # Fixing date and hour columns problems
    df['date'] = df['date'].str.replace('/','-')
    df['hour'] = df['hour'].apply(lambda x: x[:2])

    # Creating a datetime column to use as datetime index
    df['datetime'] = df['date'] + ' ' + df['hour']
    df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%d %H')

    # Setting datetime index
    df.drop(['date','hour'],axis=1,inplace=True)
    df.set_index('datetime',inplace=True)

    # Fixing numbers with comma - Pressure Data
    df['p (mB)'] = df['p (mB)'].str.replace(',','.')
    df['p (mB)'] = df['p (mB)'].astype(float)
    
    # Fixing numbers with comma - Temperature Date
    df['Temperature'] = df['Temperature'].str.replace(',','.')
    df['Temperature'] = df['Temperature'].astype(float)

    # Making '-9999' numbers a nan
    df = df.replace(-9999.0, np.nan)

    df['p (mB)'] = df['p (mB)'].interpolate()
    df['Temperature'] = df['Temperature'].interpolate()

    # Sort dataframe ( just in case )
    df.sort_index(inplace=True)

    return df

In [46]:
# Creating dataframe from all csv files
df = create_dataframe(files_path)

df

,p (mB),Temperature
datetime,,
2011-01-01 00:00:00,881.8,19.2
2011-01-01 01:00:00,882.4,19.3
2011-01-01 02:00:00,882.7,19.1
2011-01-01 03:00:00,882.6,18.1
2011-01-01 04:00:00,882.0,17.7
...,...,...
2020-12-31 19:00:00,885.2,28.1
2020-12-31 20:00:00,885.4,23.6
2020-12-31 21:00:00,885.6,23.6


Now we will delete the files that are no longer needed.

In [47]:
files_list = os.listdir()

for name in files_list:

    if name not in initial_files:

        if os.path.isdir(name):
            shutil.rmtree(name, ignore_errors=True)
        
        elif os.path.isfile(name):
            os.remove(name)

Save new dataframe cleaned and ready to use

In [48]:
df.to_csv(f"{city}_temperature_data.csv")